<a href="https://colab.research.google.com/github/luigiselmi/dl_tensorflow/blob/main/keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Keras

## Layers
A deep neural network is made of a set of stacked layers. There are many different types of layers. Layers differ mainly for the algorithms that is applied to the input data.